In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score, accuracy_score
from PIL import Image
import pandas as pd
import torchvision
import numpy as np

In [2]:
#make function for defining pathway of MRIs and read them with nibabel

def path_data_axial(category,patient):
    path = "./data/images_gray/GM/"+category+"/axial/"+patient+".png"
    return path
def path_data_sagital(category,patient):
    path = "./data/images_gray/GM/"+category+"/sagital/"+patient+".png"
    return path
def path_data_coronal(category,patient):
    path = "./data/images_gray/GM/"+category+"/coronal/"+patient+".png"
    return path

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
# Load your data
total_data = pd.read_csv("./total_data.csv")
total_data = total_data[(total_data["Research Group"] != "MCI")]
total_data = total_data.reset_index()
PID = total_data["Subject ID"]
Labels = total_data["Research Group"]

In [5]:
# Prepare your data with labeling them in to 0 and 1
data = []
new_labels = []
for label in Labels:
    if label == "CN":
        new_labels.append(0)
    elif label == "AD":
        new_labels.append(1)

In [6]:
#reading data
for i in range(len(PID)):
    img_path = path_data_axial(Labels[i], PID[i])
    data.append((img_path, new_labels[i]))


In [7]:
class ImageDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_path, label = self.data[index]
        img = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img, label

In [8]:
#transform data to tensor and do some normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [9]:
# using k-fold with k= 5

kf = KFold(n_splits=5)
fold_results = []

for fold, (train_ids, test_ids) in enumerate(kf.split(data)):
    #using subsampler for choosing more randome data
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)
    
    dataset = ImageDataset(data, transform=transform)
    
    train_loader = DataLoader(dataset, batch_size=32, sampler=train_subsampler)
    test_loader = DataLoader(dataset, batch_size=32, sampler=test_subsampler)

    # Load a pretrained ResNet-18 model
    model = torchvision.models.resnet18(pretrained=True)
    #freezing layers of the model
    for param in model.parameters():
        param.requires_grad = False
    #defining new layers
    model.fc = nn.Sequential(nn.Linear(model.fc.in_features, 512),
    #                                 nn.SELU(),
    #                                 nn.Dropout(p=0.2),
    #                                 nn.Linear(512, 512),
    #                                 nn.SELU(),
    #                                 nn.Dropout(p=0.2),
    #                                 nn.Linear(512, 3),
                                      nn.Linear(512, 128),
                                      nn.ReLU(inplace=True),
                                      nn.Dropout(p=0.2),
                                      nn.Linear(128,2),
                                      nn.ReLU(inplace=True),
                                      nn.Dropout(p=0.1),
                                      nn.Softmax(dim=1))

    model = model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    train_f1_scores, train_accuracies = [], []
    test_f1_scores, test_accuracies = [], []

    # Training Loop for this fold
    for epoch in range(50):  # Number of epochs
        model.train()
        train_targets, train_predictions = [], []
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            train_targets.extend(labels.cpu().numpy())
            train_predictions.extend(predicted.cpu().numpy())
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        train_f1 = f1_score(train_targets, train_predictions, average='weighted')
        train_acc = accuracy_score(train_targets, train_predictions)
        train_f1_scores.append(train_f1)
        train_accuracies.append(train_acc)

    model.eval()
    test_targets, test_predictions = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            test_targets.extend(labels.cpu().numpy())
            test_predictions.extend(predicted.cpu().numpy())

    test_f1 = f1_score(test_targets, test_predictions, average='weighted')
    test_acc = accuracy_score(test_targets, test_predictions)
    test_f1_scores.append(test_f1)
    test_accuracies.append(test_acc)

    print(f"Fold {fold + 1}")
    print(f"Training F1 Score: {np.mean(train_f1_scores)}, Training Accuracy: {np.mean(train_accuracies)}")
    print(f"Testing F1 Score: {test_f1}, Testing Accuracy: {test_acc}")

    fold_results.append({
        'fold': fold + 1,
        'train_f1': np.mean(train_f1_scores),
        'train_accuracy': np.mean(train_accuracies),
        'test_f1': test_f1,
        'test_accuracy': test_acc
    })

avg_train_f1 = np.mean([result['train_f1'] for result in fold_results])
avg_train_accuracy = np.mean([result['train_accuracy'] for result in fold_results])
avg_test_f1 = np.mean([result['test_f1'] for result in fold_results])
avg_test_accuracy = np.mean([result['test_accuracy'] for result in fold_results])

print(f"Average Training F1 Score: {avg_train_f1}, Average Training Accuracy: {avg_train_accuracy}")
print(f"Average Testing F1 Score: {avg_test_f1}, Average Testing Accuracy: {avg_test_accuracy}")


/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 1
Training F1 Score: 0.31372082018959796, Training Accuracy: 0.47902542372881357
Testing F1 Score: 0.4417598269022719, Testing Accuracy: 0.5932203389830508


/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 2
Training F1 Score: 0.4165060806496902, Training Accuracy: 0.5714830508474578
Testing F1 Score: 0.0795668549905838, Testing Accuracy: 0.22033898305084745


/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 3
Training F1 Score: 0.7153153661318011, Training Accuracy: 0.7470762711864406
Testing F1 Score: 0.6198830409356725, Testing Accuracy: 0.4491525423728814


/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 4
Training F1 Score: 0.2869287629543466, Training Accuracy: 0.4542372881355932
Testing F1 Score: 0.5698305084745764, Testing Accuracy: 0.6949152542372882


/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hedieh/.local/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Fold 5
Training F1 Score: 0.23285385143631981, Training Accuracy: 0.3917372881355932
Testing F1 Score: 1.0, Testing Accuracy: 1.0
Average Training F1 Score: 0.39306497627235115, Average Training Accuracy: 0.5287118644067796
Average Testing F1 Score: 0.5422080462606209, Average Testing Accuracy: 0.5915254237288136
